In [1]:
import math
import os
import torch
from pytorch_pretrained_bert import GPT2Tokenizer, GPT2LMHeadModel
# Load pre-trained model (weights)
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.eval()
# Load pre-trained model tokenizer (vocabulary)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

def get_perplexity(story):
    tokenize_input = tokenizer.encode(story)
    tensor_input = torch.tensor([tokenize_input])
    loss=model(tensor_input, lm_labels=tensor_input)
    return math.exp(loss) / len(tokenize_input)

def generate(story):
    tokenize_input = tokenizer.encode(story)
    tensor_input = torch.tensor([tokenize_input])
    result, past = model(tensor_input)
    return result,past

In [2]:
from BERT.bert import Ner

NER_model = Ner("BERT/model")

output = NER_model.predict("Steve went to Paris")

output

[{'word': 'Steve', 'tag': 'B-PER', 'confidence': 0.9989603757858276},
 {'word': 'went', 'tag': 'O', 'confidence': 0.9999730587005615},
 {'word': 'to', 'tag': 'O', 'confidence': 0.9999728202819824},
 {'word': 'Paris', 'tag': 'B-LOC', 'confidence': 0.9996751546859741}]

In [3]:
def get_mask(NER_output):
    mask = []
    for word in NER_output:
        if word['tag'] != 'O' and word['confidence'] > .9:
            mask.append(1)
        else:
            mask.append(0)
    return mask
mask = get_mask(output)
mask

[1, 0, 0, 1]

In [6]:
tokenize_input = tokenizer.encode("Steve went to Paris to buy pastries")
tensor_input = torch.tensor([tokenize_input])
loss = model(tensor_input, lm_labels=tensor_input)
math.exp(loss) / len(tokenize_input)

15.77853994447171

In [19]:
res, past = generate("Why don't we go to bed?")
res.shape

torch.Size([1, 8, 50257])